In [1]:
from __future__ import print_function, division
import numpy as np
np.random.seed(42)
import tensorflow as tf
import pdb
import fxns

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# data locations
raw_data_path = '../data/shuffled_examples'
outbase = '../output/shuffled_examples'
train_out_path = '%s/texttrain' % outbase
valid_out_path = '%s/textvalidate' % outbase
test_out_path = '%s/texttest' % outbase

# training hyperparameters
batch_size = 50
num_batches_in_train = int(434786 / batch_size)
num_epochs = 2
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

# retrieving models
global_step_to_load = None

In [3]:
tf.reset_default_graph()

# get training data
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('inputs are of size', info['seq_len'])

# get validation data
(seqv, labelv), infov = fxns.get_seq_and_label(valid_out_path)
seqv_batch, labelv_batch = tf.train.shuffle_batch([seqv, labelv],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('outputs are of size', info['label_len'])

reload(fxns)
print(test_out_path)
(seqt, labelt), info = fxns.get_seq_and_label(test_out_path, num_epochs=None)
seqt_batch, labelt_batch = tf.train.shuffle_batch([seqt, labelt],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

inputs are of size 4000
outputs are of size 24
../output/shuffled_examples/texttest


/anaconda2/lib/python2.7/site-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [6]:
print(info['tf_to_pos'])

{'HaibGm12891Pol2Pcr1x': 5, 'SydhGm12891NfkbTnfaIggrab': 12, 'SydhHuvecGata2Ucd': 16, 'HaibGm12891Pax5c20V0416101': 3, 'UtaHuvecCmyc': 20, 'UtaHuvecCtcf': 21, 'HaibGm12891Yy1sc281V0416101': 9, 'HaibHuvecPol2Pcr1x': 11, 'HaibHuvecPol24h8V0416101': 10, 'UtaGm12891Ctcf': 19, 'HaibGm12891Pol24h8Pcr1x': 4, 'UwHuvecCtcf': 23, 'HaibGm12891Pou2f2Pcr1x': 6, 'SydhHuvecMax': 17, 'UtaHuvecPol2': 22, 'BroadHuvecCtcf': 0, 'SydhHuvecCjun': 15, 'BroadHuvecPol2b': 2, 'HaibGm12891Pu1Pcr1x': 7, 'SydhGm12891Pol2Iggmus': 13, 'SydhHuvecPol2': 18, 'SydhHuvecCfosUcd': 14, 'BroadHuvecEzh239875': 1, 'HaibGm12891Taf1Pcr1x': 8}


In [7]:
# # get model -- logistic
# import logreg_model
# modelname='logreg'
# X, Y, loss, logits = logreg_model.get_logreg_model(info['seq_len'], info['label_len'])

# get model -- generic convnet
import novel_model; reload(novel_model)
task = 16
modelname='freeconv_task=%d' % task
#conv_infos = [(7,(1,20),(2,2),4),(5,(1,20),(2,2),7)]#, (8,(1,20),(2,2),5)]
conv_infos = [(14,(1,21),(2,2),2,4),(14,(1,8),(2,2),2,14),(7,(1,5),(2,2),2,14)]#, (8,(1,20),(2,2),5)]
X, Y, loss, logits = novel_model.get_novel_model(info['seq_len'], info['label_len'], conv_infos, task=task)

500.0
250.0
125.0
4000 h 15 125.0


In [8]:
# define optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate,
                                                 name='SGD-Optimizer')
    update = optimizer.minimize(loss)

# define other summaries we want
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()

In [9]:
if tf.gfile.Exists('log/' + modelname):
   tf.gfile.DeleteRecursively('log/' + modelname) 

# train

print_every = 25
save_every = 250

with tf.Session() as sess:
    
    saver = tf.train.Saver(max_to_keep=None)
    
    if global_step_to_load is None:
        global_step = 0
    else:
        saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load))
        global_step = global_step_to_load
    
#    print(layer)
    
    writer = tf.summary.FileWriter('log/' + modelname + '/train', sess.graph)
    writerv = tf.summary.FileWriter('log/' + modelname + '/valid')
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    if global_step_to_load is None:
        sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        total_loss = 0
        for i in range(num_batches_in_train):
            batch_X, batch_Y = sess.run([seq_batch, label_batch])
            batch_Xv, batch_Yv = sess.run([seqv_batch, labelv_batch])
            _, _loss = sess.run([update, loss],
                            feed_dict={X: batch_X, Y: batch_Y})
            total_loss += _loss #/ num_batches_in_train
            
            summary = sess.run(summary_op, feed_dict={X: batch_X,
                                                    Y: batch_Y})
            summaryv = sess.run(summary_op, feed_dict={X: batch_Xv,
                                                    Y: batch_Yv})
#            writer.add_summary(summary, global_step=epoch*num_batches_in_train + i)
#            writerv.add_summary(summaryv, global_step=epoch*num_batches_in_train + i)
            writer.add_summary(summary, global_step=global_step)
            writerv.add_summary(summaryv, global_step=global_step)
            
            global_step += 1
            
            if global_step % save_every == 0:
                print('saving')
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
                saver.save(sess, 'log/%s' % modelname, global_step=global_step)
                
            if global_step % print_every == 0:
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
        
        print('saving')
        print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))

        saver.save(sess, 'log/%s' % modelname, global_step=global_step)
    
    writer.close()
    coord.request_stop()
    coord.join(threads)

Epoch: 0 global_step 25 i 24 Avg loss in epoch(incomplete): 0.23134796291589738
Epoch: 0 global_step 50 i 49 Avg loss in epoch(incomplete): 0.23239789068698882
Epoch: 0 global_step 75 i 74 Avg loss in epoch(incomplete): 0.2147903320689996
Epoch: 0 global_step 100 i 99 Avg loss in epoch(incomplete): 0.21393434554338456
Epoch: 0 global_step 125 i 124 Avg loss in epoch(incomplete): 0.21064858090877533
Epoch: 0 global_step 150 i 149 Avg loss in epoch(incomplete): 0.2068044832845529
Epoch: 0 global_step 175 i 174 Avg loss in epoch(incomplete): 0.20443570928914206
Epoch: 0 global_step 200 i 199 Avg loss in epoch(incomplete): 0.2035057519003749
Epoch: 0 global_step 225 i 224 Avg loss in epoch(incomplete): 0.20492248801721466
saving
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.2069324207752943
INFO:tensorflow:log/freeconv_task=16-250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.2069324207752943
Epoc

Epoch: 0 global_step 2050 i 2049 Avg loss in epoch(incomplete): 0.20055825785892767
Epoch: 0 global_step 2075 i 2074 Avg loss in epoch(incomplete): 0.2004835409967296
Epoch: 0 global_step 2100 i 2099 Avg loss in epoch(incomplete): 0.20079160319907324
Epoch: 0 global_step 2125 i 2124 Avg loss in epoch(incomplete): 0.20074337070479112
Epoch: 0 global_step 2150 i 2149 Avg loss in epoch(incomplete): 0.2009387977351976
Epoch: 0 global_step 2175 i 2174 Avg loss in epoch(incomplete): 0.20044224320129417
Epoch: 0 global_step 2200 i 2199 Avg loss in epoch(incomplete): 0.2006887036968361
Epoch: 0 global_step 2225 i 2224 Avg loss in epoch(incomplete): 0.20079427780227715
saving
Epoch: 0 global_step 2250 i 2249 Avg loss in epoch(incomplete): 0.20055507618354426
INFO:tensorflow:log/freeconv_task=16-2250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 2250 i 2249 Avg loss in epoch(incomplete): 0.20055507618354426
Epoch: 0 global_step 2275 i 2274 Avg loss in epoch(incom

Epoch: 0 global_step 4050 i 4049 Avg loss in epoch(incomplete): 0.20008315836849772
Epoch: 0 global_step 4075 i 4074 Avg loss in epoch(incomplete): 0.2000483698137333
Epoch: 0 global_step 4100 i 4099 Avg loss in epoch(incomplete): 0.2001079008891815
Epoch: 0 global_step 4125 i 4124 Avg loss in epoch(incomplete): 0.20002212935776423
Epoch: 0 global_step 4150 i 4149 Avg loss in epoch(incomplete): 0.20000645582395865
Epoch: 0 global_step 4175 i 4174 Avg loss in epoch(incomplete): 0.19989213888248997
Epoch: 0 global_step 4200 i 4199 Avg loss in epoch(incomplete): 0.1999961799773432
Epoch: 0 global_step 4225 i 4224 Avg loss in epoch(incomplete): 0.19999795906201623
saving
Epoch: 0 global_step 4250 i 4249 Avg loss in epoch(incomplete): 0.20001704702219542
INFO:tensorflow:log/freeconv_task=16-4250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 4250 i 4249 Avg loss in epoch(incomplete): 0.20001704702219542
Epoch: 0 global_step 4275 i 4274 Avg loss in epoch(incom

Epoch: 0 global_step 6050 i 6049 Avg loss in epoch(incomplete): 0.1998682997320309
Epoch: 0 global_step 6075 i 6074 Avg loss in epoch(incomplete): 0.19972823778236354
Epoch: 0 global_step 6100 i 6099 Avg loss in epoch(incomplete): 0.19960816998156855
Epoch: 0 global_step 6125 i 6124 Avg loss in epoch(incomplete): 0.19952371465855714
Epoch: 0 global_step 6150 i 6149 Avg loss in epoch(incomplete): 0.1995372507188136
Epoch: 0 global_step 6175 i 6174 Avg loss in epoch(incomplete): 0.19937170167442275
Epoch: 0 global_step 6200 i 6199 Avg loss in epoch(incomplete): 0.19937890279196924
Epoch: 0 global_step 6225 i 6224 Avg loss in epoch(incomplete): 0.19936511580424615
saving
Epoch: 0 global_step 6250 i 6249 Avg loss in epoch(incomplete): 0.19931868604302405
INFO:tensorflow:log/freeconv_task=16-6250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 6250 i 6249 Avg loss in epoch(incomplete): 0.19931868604302405
Epoch: 0 global_step 6275 i 6274 Avg loss in epoch(inco

Epoch: 0 global_step 8050 i 8049 Avg loss in epoch(incomplete): 0.19822658697224183
Epoch: 0 global_step 8075 i 8074 Avg loss in epoch(incomplete): 0.1981840086855202
Epoch: 0 global_step 8100 i 8099 Avg loss in epoch(incomplete): 0.19818899681446728
Epoch: 0 global_step 8125 i 8124 Avg loss in epoch(incomplete): 0.19810974707282505
Epoch: 0 global_step 8150 i 8149 Avg loss in epoch(incomplete): 0.19803332779373675
Epoch: 0 global_step 8175 i 8174 Avg loss in epoch(incomplete): 0.198082517132801
Epoch: 0 global_step 8200 i 8199 Avg loss in epoch(incomplete): 0.19810383516182078
Epoch: 0 global_step 8225 i 8224 Avg loss in epoch(incomplete): 0.19803367876331676
saving
Epoch: 0 global_step 8250 i 8249 Avg loss in epoch(incomplete): 0.19797581086714158
INFO:tensorflow:log/freeconv_task=16-8250 is not in all_model_checkpoint_paths. Manually adding it.
Epoch: 0 global_step 8250 i 8249 Avg loss in epoch(incomplete): 0.19797581086714158
Epoch: 0 global_step 8275 i 8274 Avg loss in epoch(incom

Epoch: 1 global_step 10000 i 1304 Avg loss in epoch(incomplete): 0.1856095723251159
Epoch: 1 global_step 10025 i 1329 Avg loss in epoch(incomplete): 0.18507216040717256
Epoch: 1 global_step 10050 i 1354 Avg loss in epoch(incomplete): 0.1847347934926238
Epoch: 1 global_step 10075 i 1379 Avg loss in epoch(incomplete): 0.1842491864618184
Epoch: 1 global_step 10100 i 1404 Avg loss in epoch(incomplete): 0.1837845132563568
Epoch: 1 global_step 10125 i 1429 Avg loss in epoch(incomplete): 0.1836111356106984
Epoch: 1 global_step 10150 i 1454 Avg loss in epoch(incomplete): 0.18381617069372402
Epoch: 1 global_step 10175 i 1479 Avg loss in epoch(incomplete): 0.18327680766280438
Epoch: 1 global_step 10200 i 1504 Avg loss in epoch(incomplete): 0.18291438513518965
Epoch: 1 global_step 10225 i 1529 Avg loss in epoch(incomplete): 0.18301458432783488
saving
Epoch: 1 global_step 10250 i 1554 Avg loss in epoch(incomplete): 0.18323813154574758
INFO:tensorflow:log/freeconv_task=16-10250 is not in all_model_

Epoch: 1 global_step 12000 i 3304 Avg loss in epoch(incomplete): 0.18108885240390243
Epoch: 1 global_step 12025 i 3329 Avg loss in epoch(incomplete): 0.18106646606860843
Epoch: 1 global_step 12050 i 3354 Avg loss in epoch(incomplete): 0.1810373388325182
Epoch: 1 global_step 12075 i 3379 Avg loss in epoch(incomplete): 0.18113305365071583
Epoch: 1 global_step 12100 i 3404 Avg loss in epoch(incomplete): 0.18106297970791832
Epoch: 1 global_step 12125 i 3429 Avg loss in epoch(incomplete): 0.18087902037649478
Epoch: 1 global_step 12150 i 3454 Avg loss in epoch(incomplete): 0.18086679768512717
Epoch: 1 global_step 12175 i 3479 Avg loss in epoch(incomplete): 0.18075652863755395
Epoch: 1 global_step 12200 i 3504 Avg loss in epoch(incomplete): 0.18083112534831747
Epoch: 1 global_step 12225 i 3529 Avg loss in epoch(incomplete): 0.18082512206537782
saving
Epoch: 1 global_step 12250 i 3554 Avg loss in epoch(incomplete): 0.18113723021215444
INFO:tensorflow:log/freeconv_task=16-12250 is not in all_mo

Epoch: 1 global_step 14000 i 5304 Avg loss in epoch(incomplete): 0.18029910042130587
Epoch: 1 global_step 14025 i 5329 Avg loss in epoch(incomplete): 0.18019374774884273
Epoch: 1 global_step 14050 i 5354 Avg loss in epoch(incomplete): 0.18038908989037397
Epoch: 1 global_step 14075 i 5379 Avg loss in epoch(incomplete): 0.18032851471119044
Epoch: 1 global_step 14100 i 5404 Avg loss in epoch(incomplete): 0.18032017792923985
Epoch: 1 global_step 14125 i 5429 Avg loss in epoch(incomplete): 0.18024620089309484
Epoch: 1 global_step 14150 i 5454 Avg loss in epoch(incomplete): 0.1803179195446093
Epoch: 1 global_step 14175 i 5479 Avg loss in epoch(incomplete): 0.1801938762708166
Epoch: 1 global_step 14200 i 5504 Avg loss in epoch(incomplete): 0.18015381808237907
Epoch: 1 global_step 14225 i 5529 Avg loss in epoch(incomplete): 0.18008895552873774
saving
Epoch: 1 global_step 14250 i 5554 Avg loss in epoch(incomplete): 0.18013299943243835
INFO:tensorflow:log/freeconv_task=16-14250 is not in all_mod

Epoch: 1 global_step 16000 i 7304 Avg loss in epoch(incomplete): 0.1800251430271526
Epoch: 1 global_step 16025 i 7329 Avg loss in epoch(incomplete): 0.17996746659263624
Epoch: 1 global_step 16050 i 7354 Avg loss in epoch(incomplete): 0.17994697910919025
Epoch: 1 global_step 16075 i 7379 Avg loss in epoch(incomplete): 0.17989273956146465
Epoch: 1 global_step 16100 i 7404 Avg loss in epoch(incomplete): 0.179890457566659
Epoch: 1 global_step 16125 i 7429 Avg loss in epoch(incomplete): 0.1798761949948415
Epoch: 1 global_step 16150 i 7454 Avg loss in epoch(incomplete): 0.17991955289523487
Epoch: 1 global_step 16175 i 7479 Avg loss in epoch(incomplete): 0.1799391696022893
Epoch: 1 global_step 16200 i 7504 Avg loss in epoch(incomplete): 0.17987222015078388
Epoch: 1 global_step 16225 i 7529 Avg loss in epoch(incomplete): 0.1798062560505977
saving
Epoch: 1 global_step 16250 i 7554 Avg loss in epoch(incomplete): 0.17983287915631804
INFO:tensorflow:log/freeconv_task=16-16250 is not in all_model_c

In [10]:
#
# get test data

global_step_to_load_test = 17390


scores = []
labels = []

with tf.Session() as sess:
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=None)
    saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load_test))
    
    idx = 0
    while idx < 45458 / batch_size:
        batch_Xt, batch_Yt = sess.run([seqt_batch, labelt_batch])
        batch_logits = sess.run(logits, feed_dict={X: batch_Xt, Y: batch_Yt})
        scores.append(batch_logits)
        labels.append(batch_Yt)
        #print('asdf')
        idx += 1
        if idx % 100 == 0:
            print(idx)

scores_arr = np.concatenate(scores, axis=0)
labels_arr = np.concatenate(labels, axis=0)



INFO:tensorflow:Restoring parameters from log/freeconv_task=16-17390
100
200
300
400
500
600
700
800
900
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: shuffle_batch_1/random_shuffle_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch_1/random_shuffle_queue, Cast_3, Cast_2)]]


In [11]:
accuracies = np.mean((scores_arr > 0).astype(int) == labels_arr.astype(int), axis=0)

print('accuracies:')
print(accuracies)

import sklearn.metrics
print('aucs:')
aucs = [sklearn.metrics.roc_auc_score(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(aucs)

import sklearn.metrics
def prec_recall(ys_true, ys_hat):
    ys, xs, thresholds = sklearn.metrics.precision_recall_curve(ys_true, ys_hat)
    return sklearn.metrics.auc(xs, ys, reorder=True)

print('auprcs:')
auprcs = [prec_recall(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(auprcs)

props = np.mean(labels_arr, axis=0)

import json
ids = sorted(json.loads(info['tf_to_pos'].replace("'", '"')).keys())

from IPython.display import display_pretty, display_html
import pandas as pd
results = pd.DataFrame({'props':props, 'acc':accuracies, 'auprcs':auprcs, 'aucs':aucs}, index=ids).sort_values(by='aucs')
display_html(results.to_html(), raw=True)
results.to_csv('stats.' + modelname + '.tsv', index_label='id', sep='\t')

accuracies:
[0.1452967  0.28586813 0.54213187 0.9898022  0.02481319 0.97371429
 0.92454945 0.09237363 0.94112088 0.04320879 0.05828571 0.03734066
 0.94938462 0.57226374 0.09547253 0.35558242 0.94342857 0.4792967
 0.18307692 0.08004396 0.86178022 0.08347253 0.97149451 0.07402198]
aucs:
[0.44634764289845924, 0.2627052509651098, 0.4627625093360906, 0.4951204104785478, 0.6741885529610931, 0.3316056302380938, 0.4989855148669796, 0.3758948477257106, 0.3460375282201059, 0.6679994800203329, 0.5656842055666698, 0.536066639403826, 0.5182193598484133, 0.4949711345849974, 0.30348225196069323, 0.3456921540426219, 0.7399260734244194, 0.42516771019907873, 0.6490477480564097, 0.6326621980576771, 0.4641651903652819, 0.405453691856983, 0.18011829700468457, 0.45147549091221084]
auprcs:
[0.06052983309999535, 0.009296334642243812, 0.009883356551738668, 0.007514605446013673, 0.04418299893487995, 0.01812055270336776, 0.02574601324786656, 0.06748027473260094, 0.013922799812425215, 0.04025221668333389, 0.07502

,acc,aucs,auprcs,props
UtaHuvecPol2,0.971495,0.180118,0.016432,0.028505
BroadHuvecEzh239875,0.285868,0.262705,0.009296,0.011495
SydhHuvecCfosUcd,0.095473,0.303482,0.062765,0.095473
HaibGm12891Pol2Pcr1x,0.973714,0.331606,0.018121,0.026264
SydhHuvecCjun,0.355582,0.345692,0.041537,0.059978
HaibGm12891Taf1Pcr1x,0.941121,0.346038,0.013923,0.018659
HaibGm12891Pu1Pcr1x,0.092374,0.375895,0.067480,0.092374
UtaHuvecCtcf,0.083473,0.405454,0.064373,0.083473
SydhHuvecMax,0.479297,0.425168,0.016773,0.016725
BroadHuvecCtcf,0.145297,0.446348,0.060530,0.071209
